
The dataset of ancient Eurasians (see [DataS1](https://github.com/sarabehnamian/Origins-of-Ancient-Eurasian-Genomes/tree/main/steps/Step%200)). The dataset contains **genetic data** of **ancient Eurasians** from **different time periods**. The data is in the form of **SNP patterns**. The **SNP patterns** are **haplotypes** of **individuals**. Each **haplotype** is a **string** of **0s** and **1s**. The **length** of the **haplotype** is the **number of SNPs**.


Develop an interface that allows selecting **populations** and **time** and calculates the **minor allele frequencies** of the chosen populations. The program will **merge patterns** that are too close. The results will be printed on the screen as **admixture plots**.

# PLINK 1.9 

[PLINK](https://www.cog-genomics.org/plink/1.9/) is a free, open-source whole genome association analysis toolset, designed to perform a range of basic, large-scale analyses in a computationally efficient manner. The focus of PLINK is purely on analysis of genotype/phenotype data, so there is no support for a file format for storing sequence data or working with the sequence data directly.

[_More..._](https://zzz.bwh.harvard.edu/plink/)

[_Tutorial_](https://zzz.bwh.harvard.edu/plink/tutorial.shtml)

**_Installation_**

` conda install -c bioconda plink `

ver: plink-1.90b6.21 



converting the data to ped format

$plink --bfile DataS1 --maf 0.01  --recode --out s1$

the ped file is a tab-delimited text file with 6 columns:

1. Family ID
2. Individual ID
3. Paternal ID

which allells come ....


and the map file is a tab-delimited text file with 4 columns:

1. Chromosome (or contig, scaffold, etc.)
2. rs# or snp identifier
3. Genetic distance (morgans)
4. Base-pair position (bp units)
 



In [ ]:
# Load the PED file into a NumPy array
file= "/workspaces/population_genetics_MAF_admixture/Data/s1.ped"
import csv

# Open the PED file for reading
with open(file, 'r') as ped_file:
    # Create a CSV reader object
    reader = csv.reader(ped_file, delimiter=' ')
    
    # Create a dictionary to store allele counts
    allele_counts = {}
    
    # Iterate over each row in the PED file
    for row in reader:
        # write eachc row in a csv file tab delimited
        with open('s1.csv', 'a') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
        
